In [1]:
import os
import numpy as np
import pickle
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from monai.transforms import EnsureChannelFirst, LoadImage, Compose, ScaleIntensity
from monai.data import ArrayDataset
from torch.utils.data import DataLoader

import polarTransform

C:\Users\michi\Documents\Uni\_Master\_Thesis\project\venv\lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
transformer = Compose([LoadImage(image_only=True),
                       EnsureChannelFirst(),
                       ScaleIntensity()])

image_path = "data_chaksu/CHAKSU/Images/"
dm_path = "data_chaksu/CHAKSU/Disc_Masks/"

data = ArrayDataset(img=sorted([image_path + file for file in os.listdir(image_path)]),
                    img_transform=transformer,
                    seg=sorted([dm_path + file for file in os.listdir(dm_path)]),
                    seg_transform=transformer)

dataloader = DataLoader(data,
                        batch_size=1,
                        shuffle=False,
                        num_workers=2)


In [3]:
new_path = "data_chaksu_polar/CHAKSU/"
old_path = "data_chaksu/CHAKSU/"
names = sorted(os.listdir(old_path + "Images/"))

if not os.path.exists(new_path + "Disc_Masks"):
    os.makedirs(new_path + "Disc_Masks")

if not os.path.exists(new_path + "Images"):
    os.makedirs(new_path + "Images")

settings_dict = {}

for j, batch in tqdm(enumerate(dataloader)):
    image = np.array(255 * batch[0][0].permute(1, 2, 0)).astype(np.uint8)
    mask = np.array(255 * batch[1][0].permute(1, 2, 0)).astype(np.uint8)
    cup_points = np.where(mask[:, :, 1] == 255)
    center = (round(cup_points[1].mean()), round(cup_points[0].mean()))
    transformed_mask, settings = polarTransform.convertToPolarImage(mask,
                                                                    center=center,
                                                                    hasColor=True,
                                                                    radiusSize=mask.shape[0],
                                                                    angleSize=mask.shape[1])
    
    transformed_mask = transformed_mask.astype(np.uint8)
    new_mask = Image.fromarray(transformed_mask)
    new_mask.save(new_path + "Disc_Masks/" + names[j])

    transformed_image = settings.convertToPolarImage(image)
    new_image = Image.fromarray(transformed_image)
    new_image.save(new_path + "Images/" + names[j])

    settings_dict[names[j]] = settings
    

0it [00:00, ?it/s]

In [4]:
with open(new_path + "settings.pickle", "wb") as handle:
    pickle.dump(settings_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)